In [80]:
import gym
import numpy as np
import torch
import import_ipynb
import torch.autograd
import torch.optim as optim
import torch.nn as nn
from Architecture import *
from Noise import *
from Buffer import *
    

In [85]:
class Agent:
    def __init__(self,states_n, actions_n, hidden_size=256, actor_alpha=1e-4, critic_alpha=1e-3, gamma=0.99, tau=1e-2, max_memory_size=50000):
        self.hidden_size = hidden_size
        self.actor_alpha = actor_alpha
        self.critic_alpha = critic_alpha
        self.gamma = gamma
        self.tau = tau
        self.n_states = states_n
        self.n_actions = actions_n
        #create Actor Network
        
        self.Actor = Actor(self.n_states, self.hidden_size, self.n_actions)
        self.Actor_t = Actor(self.n_states, self.hidden_size, self.n_actions)
        for weight_t, weight in zip(self.Actor_t.parameters(), self.Actor.parameters()):
            weight_t.data.copy_(weight.data)      
        print(self.Actor.parameters()) 
        #self.Actor_opt = optim.Adam(self.Actor.parameters(), lr=actor_alpha)
        #create Critic Network
        self.Critic = Critic(self.n_states + self.n_actions, hidden_size, self.n_actions)
        self.Critic_t = Critic(self.n_states + self.n_actions, hidden_size, self.n_actions)
        for weight_t, weight in zip(self.Critic_t.parameters(), self.Critic.parameters()):
            weight_t.data.copy_(weight.data)
        #self.Critic_opt = optim.Adam(self.Critic.parameters(), lr=actor_alpha)
        
        #copy weights

        
            
        #create noise
        self.noise = Noise(self.n_actions)
        #create buffer
        self.replay_buffer = Buffer(max_memory_size)
        
    def push_buffer(self, state, action, reward, next_state, done):
        self.buffer.add(state, action, reward, next_state, done)
        #if buffer is full
        if(self.buffer.get_size() > batch_size):
            self.update(mini_batch_size)
    
        
    def get_action(self, state, step):
        state = torch.tensor(state.unsqueeze(0))
        #action = np.clip(action.numpy() + self.noise.get_state(), self.env.action_space.low, self.env.action_space.high)
        action = action.numpy() + self.noise.get_state()
        return action
        
    def update(self, mini_batch_size):
        states, actions, rewards, next_states, dones = self.replay_buffer.get_mini_batch(mini_batch_size)
        
        #update Critic
        next_actions = self.Actor_t(next_states)
        Q_t_next = self.Critic_t(next_states, next_actions)
        Q_targets = rewards + (gamma * Q_t_next)
        Q = self.Critic(states, actions)
        C_loss = F.mse_loss(Q, Q_targets)
        self.Critic_opt.zero_grad()
        C_loss.backward()
        self.Critic_opt.step()
        
        #update Actor
        new_actions = self.Actor(states)
        P_loss = -np.mean(self.Critic(states, new_actions))
        self.Actor_opt.zero_grad()
        P_loss.backward()
        self.Actor_opt.step()
        
        #update Target network
        for target_param, param in zip(self.Actor_t.parameters(), self.Actor.parameters()):
            target_param.data.copy_(param.data * self.tau + target_param.data * (1.0 - self.tau))
       
        for target_param, param in zip(self.Critic_t.parameters(), self.Critic.parameters()):
            target_param.data.copy_(param.data * self.tau + target_param.data * (1.0 - self.tau))
        
        
        
        
        
        